# Embed2Scale challenge demo notebook

In this notebook we show two ways of loading the challenge task data; first through our custom challenge dataloader, and then with a dataloader from the updated SSL4EO-S12 V1.1 dataset.

We then give an example of how to create a submission file by creating embeddings through random sampling.

Finally, a function for testing a submission for standard errors is provided.

Note 1: that parts of this notebook is simplified for demonstration purposes. However, the datasets and dataloaders, as well as the verification of the submission file are intended to be directly usable and true to the data and the expected submission file formats.

Note 2: The SSL4EO-S12 v1.1 dataset and the challenge dataset differs in that the SSL4EO-S12 v1.1 includes a shift in the S2 channels introduced by ESA in 2022. The Challenge data does not include this shift. Therefore, it is not recommended to use the challenge dataset, since it includes a flag to automatically add this shift to the challenge data.

In [1]:
import numpy as np
import pandas as pd

from torchvision import transforms
from torch.utils.data import DataLoader

from challenge_dataset import E2SChallengeDataset, collate_fn
from ssl4eos12_dataset import SSL4EOS12Dataset, S2L1C_MEAN, S2L1C_STD, S2L2A_MEAN, S2L2A_STD, S1GRD_MEAN, S1GRD_STD

# Configurations

In [2]:
# Order of modalities.
# In this demo, modalities are ordered the same as the default order in the SSL4EOS12 dataset class.
# Modalities are loaded in the order provided here.
# Change the order based on your needs.
modalities = ['s2l1c', 's2l2a', 's1']

# Path to challenge data folder, i.e. the folder containing the s1, s2l1c and s2l2a subfolders.
path_to_data = '/path/to/challenge/data/'

# Path to where the submission file should be written.
path_to_output_file = 'path/to/output/file.csv'

write_result_to_file = True  # Set to True to trigger saving of the csv at the end.

# Create data transformation
# Get mean and standard deviations for the modalities in the same order as the modalities.
# Note that we will use the `shift_s2_channels` flag in the challenge dataset, and we should 
# therefore use the mean and standard deviation of the SSL4EO-S12 v1.1 dataset.
mean_data = S2L1C_MEAN + S2L2A_MEAN + S1GRD_MEAN
std_data = S2L1C_STD + S2L2A_STD + S1GRD_STD

data_transform = transforms.Compose([
    # Add additional transformation here
    transforms.Normalize(mean=mean_data, std=std_data)
])

# Note that both E2SChallengeDataset and SSL4EOS12Dataset outputs torch tensors, so there is no need to a ToTensor transform.

# Load data with custom dataloader

In [3]:
# Do not concatenate modalities
# Dataloader output is {'data': {'s2l1c': s2l1c_data, 's2l2a': s2l2a_data, 's1': s1_data}, 'file_name': file_name}
# The data has shapes [n_samples, n_seasons, n_channels, height, width] (for s2l2a [1, 4, 12, 264, 264])

concatenate_modalities = False
dataset_e2s = E2SChallengeDataset(path_to_data, 
                                  modalities = modalities, 
                                  dataset_name='bands', 
                                  transform=data_transform, 
                                  concat=concatenate_modalities,
                                  output_file_name=False,
                                  shift_s2_channels=True
                                  )

# Print dataset length
print(f"Length of dataset: {len(dataset_e2s)}")

# Print shape of first sample
for m, d in dataset_e2s[0].items():
    print(f'Modality {m} shape:', d.shape)

Length of dataset: 5149
Modality s2l1c shape: torch.Size([1, 4, 13, 264, 264])
Modality s2l2a shape: torch.Size([1, 4, 12, 264, 264])
Modality s1 shape: torch.Size([1, 4, 2, 264, 264])


In [4]:
# Concatenate modalities
# dataloader output is {'data': concatenated_data, 'file_name': file_name}
# The data has shapes [n_samples, n_seasons, n_channels, height, width] (for concatenated_data [1, 4, 27, 264, 264])

concatenate_modalities = True
dataset_e2s = E2SChallengeDataset(path_to_data, 
                                  modalities = modalities, 
                                  dataset_name='bands', 
                                  transform=None,#data_transform, 
                                  concat=concatenate_modalities,
                                  output_file_name=True,
                                  shift_s2_channels=True
                                 ) 

# Print dataset length
print(f"Length of dataset: {len(dataset_e2s)}")

# Print shape of first sample
print(dataset_e2s[0]['data'].shape)

Length of dataset: 5149
torch.Size([1, 4, 27, 264, 264])


# Load with SSL4EOS12 V1.1 dataloader

Note that we have modified the code to allow for a different number of samples per zarr files. The challenge task data consists of a single sample per file, while SSL4EO-S12 V1.1 has 64 samples per zarr file.

Note that the SSL4EO-S12 v1.1 dataset and the challenge dataset differs in that the SSL4EO-S12 v1.1 includes a shift in the S2 channels introduced by ESA in 2022. The Challenge data does not include this shift. Therefore, it is not recommended to use the challenge dataset, since it includes a flag to automatically add this shift to the challenge data.

In [5]:
dataset_ssl4eo = SSL4EOS12Dataset(
    data_dir=path_to_data,
    modalities=modalities, # optional, list of modality folders.
    transform=None,#data_transform,  # optional, torchvision transforms. Returns tensors if not provided.
    concat=True,  # Concatenate all modalities along the band dimension.
    single_timestamp=False,  # Load single timestamps rather than time series.
    num_batch_samples=1,  # optional, subsample samples in each zarr file.
    num_timestamps=4  # optional, number of seasons to include
)

# Print dataset length
print(f"Length of dataset: {len(dataset_ssl4eo)}")

# Print shape of first sample
print(dataset_ssl4eo[0].shape)

Length of dataset: 5149
torch.Size([1, 4, 27, 264, 264])


In [6]:
# Compare the output from the datasets
print("The two datasets' first sample is the same:", np.all((dataset_e2s[0]['data'] == dataset_ssl4eo[0]).numpy()))

# Shift the data loaded with the SSL4EO-S12 dataset by 1000 and show they 
# are now identical to the challenge dataset with the `shift_s2_channels` flag set to True
shifted_data = dataset_ssl4eo[0]
shifted_data[:, :, :-2, :, :] += 1000
print("The two datasets' first sample is the same after shifting:", np.all((dataset_e2s[0]['data'] == shifted_data).numpy()))

The two datasets' first sample is the same: False
The two datasets' first sample is the same after shifting: True


# Dataloader

In [7]:
train_loader  = DataLoader(
    dataset=dataset_e2s,
    batch_size=2,  # Note that each each challenge task zarr file contains a single sample.
    shuffle=True,
    collate_fn=collate_fn,  # Data needs to be concatenated along sample dimension instead of being stacked.
)

for ind, data_file_name in enumerate(train_loader):
    for find, fn in enumerate(data_file_name['file_name']):
        print(f'File name {find}:', fn[0:10] + '...')
    print(data_file_name['data'].shape)
    break

File name 0: d02fd64ab4...
File name 1: 7642bfbee4...
torch.Size([2, 4, 27, 264, 264])


# Create submission file

In this section, we create a submission by randomly generating embeddings of the correct size.
Finally, we create a submission file.

We use the E2SChallengeDataset since we can easily get the sample ID (file name) from the dataloader.

In [8]:
def create_submission_from_dict(emb_dict):
    """Assume dictionary has format {hash-id0: embedding0, hash-id1: embedding1, ...}
    """
    df_submission = pd.DataFrame.from_dict(emb_dict, orient='index')

    # Reset index with name 'id'
    df_submission.index.name = 'id'
    df_submission.reset_index(drop=False, inplace=True)
        
    return df_submission
        

In [9]:
# Randomly generate embeddings from normal distribution.

embedding_dim = 1024
embeddings = {}
rng = np.random.default_rng(seed=None)

# Allow stop early for demonstration purposes
create_n_embeddings = np.inf # 10 # np.inf

# Create random embeddings before the loop to speed up this example
rand_embds = rng.normal(0, 1, size=(len(dataset_e2s), embedding_dim))

for ind, data_file_name in enumerate(dataset_e2s):
    # -------------------------
    # Do compression magic here
    # -------------------------
    
    file_name = data_file_name['file_name']
    
    # Insert the random embeddings
    emb = rand_embds[ind, :]
    embeddings[file_name] = emb

    # Stop early in this example
    if ind >= create_n_embeddings-1:
        break

In [10]:
# Create submission file
submission_file = create_submission_from_dict(embeddings)

In [11]:
print('Number of embeddings:', len(submission_file))

Number of embeddings: 5149


In [12]:
submission_file.head()

,id,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,fec24d0cda8793ff55e1059c7b88763fee8d58d3decf78...,0.334640,0.349985,-2.320028,-2.047363,-2.303194,-2.228242,-0.014234,-0.337983,0.108561,...,-0.639381,1.196623,0.129239,-0.376327,-0.089538,0.722393,-1.291948,0.964103,-1.093549,-0.137151
1,67960f4c8870a8aa52f295da0f0fea6d708c3cee2555a4...,-0.478898,-0.334614,-1.030222,0.162082,0.269054,0.436804,-0.765836,-0.600234,-0.537268,...,-1.167944,0.027686,-0.718442,0.670432,-0.017366,-0.898831,1.177437,0.829586,0.659521,-0.915120
2,9688abfaebaea5dca2ec8bde771a7bf1e2bba8e661b777...,0.041639,-1.587280,-0.465861,0.512501,2.433723,-0.935121,-1.598009,1.420454,-1.963545,...,2.058060,-0.065768,-0.015405,0.226882,-0.783073,-0.506375,-0.025987,1.707453,-0.357952,0.122508
3,fa3ae237ee6e2ee569c20a1e088112cf2105300d9272cc...,0.507184,0.614263,-0.629493,-1.052323,0.573738,-0.128368,2.309440,-0.303956,-1.870584,...,0.372565,0.260517,0.269054,-0.485850,-0.915676,-1.325099,0.278023,1.422932,-0.109799,-1.695412
4,430590d31e38c5b345a92dc7d9eb8d126c01abced0cf1a...,-1.363808,-0.505722,0.168191,-1.367364,-0.512147,-0.261059,0.588672,0.191185,-1.364636,...,0.935763,0.238390,1.119201,0.566553,1.474977,1.870136,-0.550028,-1.508855,-1.106044,-0.980552


In [13]:
# Write submission
if write_result_to_file:
    submission_file.to_csv(path_to_output_file, index=False)

# Verify submission file integrity

Below we provide a snippet from a function which will read your embeddings and test for the same errors that the evaluation will check for. The function is similar to how the submission files are loaded.

The intention of this function is to help to verify that a submission has the right structure and contents, check for missing embeddings or NaN values, prior to submission.

The function is intended to be a support. Successfully completing this function does not guarantee fault-free submission files, but is an indication that the most common errors are not present.

In [14]:
def test_submission(path_to_submission: str, 
                    expected_embedding_ids: set, 
                    embedding_dim: int = 1024):
    # Load data
    df = pd.read_csv(path_to_submission, header=0)

    # Verify that id is in columns
    if 'id' not in df.columns:
        raise ValueError(f"""Submission file must contain column 'id'.""")

    # Temporarily set index to 'id'
    df.set_index('id', inplace=True)

    # Check that all samples are included
    submitted_embeddings = set(df.index.to_list())
    n_missing_embeddings = len(expected_embedding_ids.difference(submitted_embeddings))
    if n_missing_embeddings > 0:
        raise ValueError(f"""Submission is missing {n_missing_embeddings} embeddings.""")
    
    # Check that embeddings have the correct length
    if len(df.columns) != embedding_dim:
        raise ValueError(f"""{embedding_dim} embedding dimensions, but provided embeddings have {len(df.columns)} dimensions.""")

    # Convert columns to float
    try:
        for col in df.columns:
            df[col] = df[col].astype(float)
    except Exception as e:
        raise ValueError(f"""Failed to convert embedding values to float.
    Check embeddings for any not-allowed character, for example empty strings, letters, etc.
    Original error message: {e}""")

    # Check if any NaNs 
    if df.isna().any().any():
        raise ValueError(f"""Embeddings contain NaN values.""")
    
    # Successful completion of the function
    return True

In [15]:
# We use the created embeddings as the list of all samples.
# This can be done since we are sure to have fully looped through the dataset.
# A better way would be to find all the IDs in the challenge data separately, e.g. from the dataloader.
embedding_ids = set(embeddings.keys())

# Test submission
assert test_submission(path_to_output_file, embedding_ids, embedding_dim)